# Data Collection

Imports

In [26]:
import pandas as pd
import numpy as np

import time
import retrying

In [2]:
import spotipy
from spotipy import SpotifyClientCredentials, util

I created an app on Spotify's developer page and was retrieved my client ID and client secret. I ran a cell with these codes and deleted them after for security purposes. I you want to replicate this notebook, you can create an app and replace `CLIENT_ID` and `CLIENT_SECRET` with your app's in the code below.

In [4]:
creds= SpotifyClientCredentials(client_id= CLIENT_ID, client_secret= CLIENT_SECRET)

In [5]:
sp = spotipy.Spotify(client_credentials_manager= creds)

In [6]:
# searched for largest playlist on spotify
# it has 10,000 songs of many genres/song types
playlist_link = "https://open.spotify.com/playlist/5S8SJdl1BDc0ugpkEvFsIL"

# extracting playlist URI
playlist_URI = playlist_link.split("/")[-1]

# make a list of all songs' uri 
song_uri_list = [song["track"]["uri"] for song in sp.playlist_tracks(playlist_URI)["items"]]
len(song_uri_list)

100

We can see there is a limit on how many songs you can get using the `sp.playlist_tracks()` function, so we will create a for loop to change the offset(which index to start) to collect all 10,000 songs.

In [7]:
song_uri_list=[]

# offsets that skip by 100
offsets= [*range(0, 10_000, 100)]

In [8]:
for o in offsets:
    song_uris= [song["track"]["uri"] for song in sp.playlist_tracks(playlist_URI, offset= o)["items"]]
    song_uri_list.extend(song_uris)
len(song_uri_list)

10000

We have all 10,000 songs now.

### Getting all songs into a dataframe orientation

In [17]:
sp.audio_features(song_uri_list[0])

[{'danceability': 0.615,
  'energy': 0.779,
  'key': 2,
  'loudness': -6.454,
  'mode': 1,
  'speechiness': 0.135,
  'acousticness': 0.0665,
  'instrumentalness': 0,
  'liveness': 0.155,
  'valence': 0.453,
  'tempo': 160.0,
  'type': 'audio_features',
  'id': '7zgqtptZvhf8GEmdsM2vp2',
  'uri': 'spotify:track:7zgqtptZvhf8GEmdsM2vp2',
  'track_href': 'https://api.spotify.com/v1/tracks/7zgqtptZvhf8GEmdsM2vp2',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/7zgqtptZvhf8GEmdsM2vp2',
  'duration_ms': 208198,
  'time_signature': 4}]

In [19]:
audio_data= sp.audio_features(song_uri_list[0])

danceability = audio_data[0]['danceability']
energy = audio_data[0]['energy']
key = audio_data[0]['key']
loudness = audio_data[0]['loudness']
mode = audio_data[0]['mode']
speechiness = audio_data[0]['speechiness']
acousticness = audio_data[0]['acousticness']
instrumentalness = audio_data[0]['instrumentalness']
liveness = audio_data[0]['liveness']
valence = audio_data[0]['valence']
tempo = audio_data[0]['tempo']
duration_ms = audio_data[0]['duration_ms']
time_signature = audio_data[0]['time_signature']

In [20]:
audio_data = pd.DataFrame({
    'danceability': [danceability],
    'energy': [energy],
    'key': [key],
    'loudness': [loudness],
    'mode': [mode],
    'speechiness': [speechiness],
    'acousticness': [acousticness],
    'instrumentalness': [instrumentalness],
    'liveness': [liveness],
    'valence': [valence],
    'tempo': [tempo],
    'duration': [duration_ms],
    'time signature': [time_signature]
})

In [21]:
audio_data.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration,time signature
0,0.615,0.779,2,-6.454,1,0.135,0.0665,0,0.155,0.453,160.0,208198,4


In [16]:
sp.track(song_uri_list[0])

{'album': {'album_type': 'single',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/06HL4z0CvFAxyc27GXpf02'},
    'href': 'https://api.spotify.com/v1/artists/06HL4z0CvFAxyc27GXpf02',
    'id': '06HL4z0CvFAxyc27GXpf02',
    'name': 'Taylor Swift',
    'type': 'artist',
    'uri': 'spotify:artist:06HL4z0CvFAxyc27GXpf02'}],
  'available_markets': [],
  'external_urls': {'spotify': 'https://open.spotify.com/album/0HG8fMDhvN2tH5uPHFsyZP'},
  'href': 'https://api.spotify.com/v1/albums/0HG8fMDhvN2tH5uPHFsyZP',
  'id': '0HG8fMDhvN2tH5uPHFsyZP',
  'images': [{'height': 640,
    'url': 'https://i.scdn.co/image/ab67616d0000b2734322e9bd7d57d061d0e19e1f',
    'width': 640},
   {'height': 300,
    'url': 'https://i.scdn.co/image/ab67616d00001e024322e9bd7d57d061d0e19e1f',
    'width': 300},
   {'height': 64,
    'url': 'https://i.scdn.co/image/ab67616d000048514322e9bd7d57d061d0e19e1f',
    'width': 64}],
  'name': '...Ready For It?',
  'release_date': '2017-09-03',
  'rele

In [12]:
song_data= sp.track(song_uri_list[0])

artist_name = song_data['artists'][0]['name']
track_name = song_data['name']
album_name = song_data['album']['name']
release_date = song_data['album']['release_date']
popularity = song_data['popularity']

In [13]:
song_data = pd.DataFrame({
    'artist': [artist_name],
    'track': [track_name],
    'album': [album_name],
    'release_date': [release_date],
    'popularity': [popularity]
})
song_data.head()

,artist,track,album,release_date,popularity
0,Taylor Swift,...Ready For It?,...Ready For It?,2017-09-03,0


We will now make functions to do what we did for one song above for all the songs.

In [29]:
@retrying.retry(stop_max_attempt_number=2, wait_fixed=2000)

def get_song_data(song, sp):
    
    '''
    This function takes in a song uri and your spotipy.Spotify() function saved as a variable.
    The function extracts the data from spotipy's .track() and .audio_features() and saves them into a dictionary.
    The funtions this dictionaries containing all the song's info. 
    '''
    
    song_data = sp.track(song)
    audio_data = sp.audio_features(song)

    song_feat = {
                
                # from song data 
                'song': song_data['name'],
                'artist': song_data['artists'][0]['name'],
                'album': song_data['album']['name'],
                'release_date': song_data['album']['release_date'],
                
                # from audio data
                'danceability': audio_data[0]['danceability'],
                'energy': audio_data[0]['energy'],
                'key': audio_data[0]['key'],
                'loudness': audio_data[0]['loudness'],
                'mode': audio_data[0]['mode'],
                'speechiness': audio_data[0]['speechiness'],
                'acousticness': audio_data[0]['acousticness'],
                'instrumentalness': audio_data[0]['instrumentalness'],
                'liveness': audio_data[0]['liveness'],
                'valence': audio_data[0]['valence'],
                'tempo': audio_data[0]['tempo'],
                'duration': audio_data[0]['duration_ms'],
                'time_signature': audio_data[0]['time_signature']
            }

    return song_feat

In [38]:
# testing
get_song_data(song_uri_list[0], sp)

{'song': '...Ready For It?',
 'artist': 'Taylor Swift',
 'album': '...Ready For It?',
 'release_date': '2017-09-03',
 'danceability': 0.615,
 'energy': 0.779,
 'key': 2,
 'loudness': -6.454,
 'mode': 1,
 'speechiness': 0.135,
 'acousticness': 0.0665,
 'instrumentalness': 0,
 'liveness': 0.155,
 'valence': 0.453,
 'tempo': 160.0,
 'duration': 208198,
 'time_signature': 4}

In [30]:
def get_song_features(song_uri_list, sp):
    
    '''
    This function takes in a list of song uri's and your spotipy.Spotify() function saved as a variable.
    The function goes through each song's uri and uses the get_song_data() function to extracts the data from each song.
    It then saves each song's feature dictionary into a list. 
    A sleep function is used to prevent 429 API rate limit errors.
    The funtions returns a list of dictionaries containing all the song's info. 
    '''
    
    song_feats = []

    for song in song_uri_list:
        try:
            song_feat = get_song_data(song, sp)
            song_feats.append(song_feat)
            time.sleep(2)
        except:
            print(f"Failed to retrieve data for song with URI: {song}")
    
    return song_feats

In [31]:
all_song_feats= get_song_features(song_uri_list, sp)

Failed to retrieve data for song with URI: spotify:local:::In+the+Shadow+of+the+Valley:187
Failed to retrieve data for song with URI: spotify:local:::April+Sweatpants:166
Failed to retrieve data for song with URI: spotify:local:SNSD::Gee+%28Korean+Ver.+MP3+only%29:205
Failed to retrieve data for song with URI: spotify:local:::Hero:194
Failed to retrieve data for song with URI: spotify:local:::Beneath+the+Mask+%5BWith+Lyrics%5D+-+Persona+5:285
Failed to retrieve data for song with URI: spotify:local:::Last+Surprise+%5BWith+Lyrics%5D+-+Persona+5:236
Failed to retrieve data for song with URI: spotify:local:::Vampire+Weekend++Ottoman:244
Failed to retrieve data for song with URI: spotify:local:::Vampire+Weekend+-+Ottoman:234
Failed to retrieve data for song with URI: spotify:local:Harry+Gregson-Williams:The+Chronicles+of+Narnia%3A+The+Lion%2C+The+Witch+and+the+Wardrobe+%28Soundtrack+from+the+Motion+Picture%29:Father+Christmas:200
Failed to retrieve data for song with URI: spotify:local:Har

There were a few songs that weren't able to be retrieved, but most were. We will make a dataframe with all the songs from the list of dictionaries.

### Create Dataframe

In [32]:
song_df= pd.DataFrame(all_song_feats)

In [33]:
song_df.head()

,song,artist,album,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration,time_signature
0,...Ready For It?,Taylor Swift,...Ready For It?,2017-09-03,0.615,0.779,2,-6.454,1,0.1350,0.0665,0.000000,0.1550,0.453,160.000,208198,4
1,Life Changes,Thomas Rhett,Life Changes,2017-09-08,0.687,0.845,7,-4.370,1,0.0576,0.1000,0.000000,0.0452,0.809,87.972,190227,4
2,24K Magic,Bruno Mars,24K Magic,2016-11-17,0.818,0.803,1,-4.282,1,0.0797,0.0340,0.000000,0.1530,0.632,106.970,225983,4
3,Galway Girl,Ed Sheeran,÷ (Deluxe),2017-03-03,0.624,0.876,9,-3.374,1,0.1000,0.0735,0.000000,0.3270,0.781,99.943,170827,4
4,Photograph,Ed Sheeran,x (Deluxe Edition),2014-06-21,0.614,0.379,4,-10.480,1,0.0476,0.6070,0.000464,0.0986,0.201,107.989,258987,4


In [35]:
song_df.tail()

,song,artist,album,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration,time_signature
9956,Funeral For A Friend / Love Lies Bleeding,Elton John,Goodbye Yellow Brick Road (40th Anniversary Ce...,1973-10-05,0.410,0.761,9,-8.507,0,0.0465,0.01980,0.08470,0.2470,0.193,138.712,666572,4
9957,Fool's Overture,Supertramp,Even In The Quietest Moments,1977-01-01,0.406,0.306,3,-10.482,1,0.0372,0.31300,0.00790,0.0727,0.073,135.272,652560,4
9958,Heart of the Sunrise - 2003 Remaster,Yes,Fragile (Deluxe Edition),1971-11-26,0.362,0.507,1,-11.229,1,0.0394,0.01740,0.21600,0.1130,0.456,146.641,634440,3
9959,Venus And Mars / Rock Show / Jet - Live / Rema...,Paul McCartney,Wings Over America (Remastered),1976-12-10,0.331,0.733,2,-8.671,1,0.0468,0.08870,0.00174,0.9470,0.380,128.512,620747,4
9960,Mr. Brightside,The Killers,Hot Fuss,2004-06-15,0.342,0.932,1,-3.644,1,0.0868,0.00106,0.00000,0.0938,0.226,148.148,222107,4


In [34]:
len(song_df)

9961

It looks like we lost only 39 songs, so we still have a good number of songs. Now we can move onto cleaning and EDA.

### Create csv file

In [36]:
# save to csv for use in our eda and modeling
song_df.to_csv('../data/spotify_songs.csv', index=False)